# Chapter 9

*Modeling and Simulation in Python*

Copyright 2021 Allen Downey

License: [Creative Commons Attribution-NonCommercial-ShareAlike 4.0 International](https://creativecommons.org/licenses/by-nc-sa/4.0/)

In [ ]:
# check if the libraries we need are installed

try:
    import pint
except ImportError:
    !pip install pint
    import pint
    
try:
    from modsim import *
except ImportError:
    !pip install modsimpy
    from modsim import *

The following displays SymPy expressions and provides the option of showing results in LaTeX format.

In [ ]:
from sympy.printing import latex

def show(expr, show_latex=False):
    """Display a SymPy expression.
    
    expr: SymPy expression
    show_latex: boolean
    """
    if show_latex:
        print(latex(expr))
    return expr

### Analysis with SymPy

Create a symbol for time.

In [ ]:
import sympy as sp

t = sp.symbols('t')
t

If you combine symbols and numbers, you get symbolic expressions.

In [ ]:
expr = t + 1
expr

The result is an `Add` object, which just represents the sum without trying to compute it.

In [ ]:
type(expr)

`subs` can be used to replace a symbol with a number, which allows the addition to proceed.

In [ ]:
expr.subs(t, 2)

`f` is a special class of symbol that represents a function.

In [ ]:
f = sp.Function('f')
f

The type of `f` is `UndefinedFunction`

In [ ]:
type(f)

SymPy understands that `f(t)` means `f` evaluated at `t`, but it doesn't try to evaluate it yet.

In [ ]:
f(t)

`diff` returns a `Derivative` object that represents the time derivative of `f`

In [ ]:
dfdt = sp.diff(f(t), t)
dfdt

In [ ]:
type(dfdt)

We need a symbol for `alpha`

In [ ]:
alpha = sp.symbols('alpha')
alpha

Now we can write the differential equation for proportional growth.

In [ ]:
eq1 = sp.Eq(dfdt, alpha*f(t))
eq1

And use `dsolve` to solve it.  The result is the general solution.

In [ ]:
solution_eq = sp.dsolve(eq1)
solution_eq

We can tell it's a general solution because it contains an unspecified constant, `C1`.

In this example, finding the particular solution is easy: we just replace `C1` with `p_0`

In [ ]:
C1, p_0 = sp.symbols('C1 p_0')

In [ ]:
particular = solution_eq.subs(C1, p_0)
particular

In the next example, we have to work a little harder to find the particular solution.

### Solving the quadratic growth equation 

We'll use the (r, K) parameterization, so we'll need two more symbols:

In [ ]:
r, K = sp.symbols('r K')

Now we can write the differential equation.

In [ ]:
eq2 = sp.Eq(sp.diff(f(t), t), r * f(t) * (1 - f(t)/K))
eq2

And solve it.

In [ ]:
solution_eq = sp.dsolve(eq2)
solution_eq

The result, `solution_eq`, contains `rhs`, which is the right-hand side of the solution.

In [ ]:
general = solution_eq.rhs
general

We can evaluate the right-hand side at $t=0$

In [ ]:
at_0 = general.subs(t, 0)
at_0

Now we want to find the value of `C1` that makes `f(0) = p_0`.

So we'll create the equation `at_0 = p_0` and solve for `C1`.  Because this is just an algebraic identity, not a differential equation, we use `solve`, not `dsolve`.

The result from `solve` is a list of solutions.  In this case, [we have reason to expect only one solution](https://en.wikipedia.org/wiki/Picard%E2%80%93Lindel%C3%B6f_theorem), but we still get a list, so we have to use the bracket operator, `[0]`, to select the first one.

In [ ]:
solutions = sp.solve(sp.Eq(at_0, p_0), C1)
type(solutions), len(solutions)

In [ ]:
value_of_C1 = solutions[0]
value_of_C1

Now in the general solution, we want to replace `C1` with the value of `C1` we just figured out.

In [ ]:
particular = general.subs(C1, value_of_C1)
particular

The result is complicated, but SymPy provides a method that tries to simplify it.

In [ ]:
particular = sp.simplify(particular)
particular

Often simplicity is in the eye of the beholder, but that's about as simple as this expression gets.

Just to double-check, we can evaluate it at `t=0` and confirm that we get `p_0`

In [ ]:
particular.subs(t, 0)

This solution is called the [logistic function](https://en.wikipedia.org/wiki/Population_growth#Logistic_equation).

In some places you'll see it written in a different form:

$f(t) = \frac{K}{1 + A e^{-rt}}$

where $A = (K - p_0) / p_0$.

We can use SymPy to confirm that these two forms are equivalent.  First we represent the alternative version of the logistic function:

In [ ]:
A = (K - p_0) / p_0
A

In [ ]:
logistic = K / (1 + A * sp.exp(-r*t))
logistic

To see whether two expressions are equivalent, we can check whether their difference simplifies to 0.

In [ ]:
sp.simplify(particular - logistic)

This test only works one way: if SymPy says the difference reduces to 0, the expressions are definitely equivalent (and not just numerically close).

But if SymPy can't find a way to simplify the result to 0, that doesn't necessarily mean there isn't one.  Testing whether two expressions are equivalent is a surprisingly hard problem; in fact, there is no algorithm that can solve it in general.

### Exercises

**Exercise:** Solve the quadratic growth equation using the alternative parameterization

$\frac{df(t)}{dt} = \alpha f(t) + \beta f^2(t) $

In [ ]:
# Solution goes here

In [ ]:
# Solution goes here

In [ ]:
# Solution goes here

In [ ]:
# Solution goes here

In [ ]:
# Solution goes here

In [ ]:
# Solution goes here

In [ ]:
# Solution goes here

**Exercise:**  Use [WolframAlpha](https://www.wolframalpha.com/) to solve the quadratic growth model, using either or both forms of parameterization:

    df(t) / dt = alpha f(t) + beta f(t)^2

or

    df(t) / dt = r f(t) (1 - f(t)/K)

Find the general solution and also the particular solution where `f(0) = p_0`.